In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#def load_data():
global smart
global gas
global weather
df =pd.read_csv("/datc/opschaler/smartmeter_data/P01S01W0373.csv",header=0,delimiter=";",parse_dates=['Timestamp', 'gasTimestamp'])

# Split the smartmeter and gasmeter data
smart = df.iloc[:,:7]
gas = df.iloc[:, 7:]

#Read in weather data
#weather_path = "//datc//opschaler//weather_data//20170101to20180101.txt"
#weather_column_names = ['STN','YYYYMMDD','HH','DD','FH','FF','FX','T','T10','TD','SQ','Q','DR','RH','P','VV','N','U','WW','IX','M','R','S','O','Y']
#weather = pd.read_csv(weather_path, delimiter=',', comment='#', names=weather_column_names, parse_dates=['YYYYMMDD', 'HH'])

#weather['T10'] = weather['T10'].str.strip()#remove blank spaces in the whole column
#weather['T10'].replace('',0,inplace=True)#replace empty gaps with 0 value(int)
#weather['T10']=weather.T10.astype(float)

#weather['T'] = weather['T']*0.1 # Convert temperature to celcius instead of 0.1 celcius
#weather['HH'] = weather['HH']+':00:00'
#weather['HH'] = weather['HH'].str.strip() # removes the blank spaces infront of the strings
#weather['HH'] = weather['HH'].str.replace('24:00:00', '23:59:59') # KNMI uses 24 for 00 hours.... to have to convert this for to_datetime to be able to work
#weather.loc[:,'datetime'] = pd.to_datetime(weather['YYYYMMDD'].astype(str)+' '+weather['HH'].astype(str)) #create datetime column


In [3]:
"""
Problem: How to merge 3 frames with 3 different datetime's?
The format is the same, but they don't have ''exact'' matches with each other. 
i.e. 

"""
#load_data()

# Change all datetime columns to the same column name
smart = smart.rename(index=str, columns={"Timestamp":"datetime"})
gas = gas.rename(index=str, columns={"gasTimestamp":"datetime"})


#df = df[['datetime', 'eMeter', 'eMeterLow', 'ePower', 'gasMeter', 'T', 'Q']]
#df = df[['datetime', 'eMeter', 'eMeterLow', 'ePower', 'gasMeter']]
smart.head()

#del weather['STN'], weather['HH'], weather['YYYYMMDD']

# Set datetime as index
#weather = weather.set_index(['datetime'])
smart = smart.set_index(['datetime'])
gas = gas.set_index(['datetime'])

In [4]:
"""
Resample the dataframes to 10s intervals.
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.resample.html
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.interpolate.html#pandas.DataFrame.interpolate
"""
############################
#weather = weather.resample('10s').interpolate(method='time')
############################
smart = smart.resample('10s').mean()
# gas contains duplicates, they need to be removed
#gas = gas.drop_duplicates()
gas = gas.resample('H').mean() # Does not look nice, but it works

# Create a ''gasPower'' column. This equals the gas used in that hour
#gas['gasPower'] = gas['gasMeter'].diff()
# First entry is NaN, replace NaN by second entry
#gas['gasPower'][0] = gas['gasPower'][1]
#gas = gas.resample('10s').interpolate(method='time')

In [5]:
#df = pd.merge(smart,gas,left_index=True, right_index=True)

In [6]:
smart.head()

,eMeter,eMeterReturn,eMeterLow,eMeterLowReturn,ePower,ePowerReturn
datetime,,,,,,
2017-03-07 18:14:20,3717.472,0.0,3883.382,0.0,376.0,0.0
2017-03-07 18:14:30,3717.473,0.0,3883.382,0.0,381.0,0.0
2017-03-07 18:14:40,3717.474,0.0,3883.382,0.0,378.0,0.0
2017-03-07 18:14:50,3717.475,0.0,3883.382,0.0,376.0,0.0
2017-03-07 18:15:00,3717.476,0.0,3883.382,0.0,380.0,0.0


In [7]:
gas.head()

,gasMeter
datetime,
2017-03-07 18:00:00,1599.792
2017-03-07 19:00:00,1599.986
2017-03-07 20:00:00,1600.179
2017-03-07 21:00:00,1600.367
2017-03-07 22:00:00,1600.513


# Working with electricity dataframe

In [8]:
#First check how many nans are there
smart_nan=smart.isnull().any(axis=1)
print(smart[smart_nan].info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2947 entries, 2017-03-07 18:16:10 to 2017-03-28 23:50:30
Data columns (total 6 columns):
eMeter             0 non-null float64
eMeterReturn       0 non-null float64
eMeterLow          0 non-null float64
eMeterLowReturn    0 non-null float64
ePower             0 non-null float64
ePowerReturn       0 non-null float64
dtypes: float64(6)
memory usage: 161.2 KB
None


In [9]:
print(smart[smart_nan])

                     eMeter  eMeterReturn  eMeterLow  eMeterLowReturn  ePower  \
datetime                                                                        
2017-03-07 18:16:10     NaN           NaN        NaN              NaN     NaN   
2017-03-07 18:17:40     NaN           NaN        NaN              NaN     NaN   
2017-03-07 18:46:00     NaN           NaN        NaN              NaN     NaN   
2017-03-07 18:47:20     NaN           NaN        NaN              NaN     NaN   
2017-03-07 19:17:10     NaN           NaN        NaN              NaN     NaN   
2017-03-07 19:18:30     NaN           NaN        NaN              NaN     NaN   
2017-03-07 19:48:10     NaN           NaN        NaN              NaN     NaN   
2017-03-07 19:49:30     NaN           NaN        NaN              NaN     NaN   
2017-03-07 20:18:00     NaN           NaN        NaN              NaN     NaN   
2017-03-07 20:19:20     NaN           NaN        NaN              NaN     NaN   
2017-03-07 20:49:10     NaN 

In [10]:
#Then just interpolate
smart_final=smart.interpolate('time')
print(smart_final)

                        eMeter  eMeterReturn  eMeterLow  eMeterLowReturn  \
datetime                                                                   
2017-03-07 18:14:20  3717.4720           0.0   3883.382              0.0   
2017-03-07 18:14:30  3717.4730           0.0   3883.382              0.0   
2017-03-07 18:14:40  3717.4740           0.0   3883.382              0.0   
2017-03-07 18:14:50  3717.4750           0.0   3883.382              0.0   
2017-03-07 18:15:00  3717.4760           0.0   3883.382              0.0   
2017-03-07 18:15:10  3717.4770           0.0   3883.382              0.0   
2017-03-07 18:15:20  3717.4780           0.0   3883.382              0.0   
2017-03-07 18:15:30  3717.4790           0.0   3883.382              0.0   
2017-03-07 18:15:40  3717.4800           0.0   3883.382              0.0   
2017-03-07 18:15:50  3717.4810           0.0   3883.382              0.0   
2017-03-07 18:16:00  3717.4820           0.0   3883.382              0.0   
2017-03-07 1

# Working with gas dataframe

In [11]:
#
gas_nan=gas.isnull().any(axis=1)
print(gas[gas_nan])

                     gasMeter
datetime                     
2017-03-16 17:00:00       NaN
2017-03-26 02:00:00       NaN
2017-03-26 04:00:00       NaN


In [12]:
gas_final = gas.resample('10s').interpolate(method='time')
# Create a ''gasPower'' column. This equals the gas used in that hour
gas_final['gasPower'] = gas_final['gasMeter'].diff()
# First entry is NaN, replace NaN by second entry
gas_final['gasPower'][0] = gas_final['gasPower'][1]
#gas = gas.resample('10s').interpolate(method='time')
print(gas_final)

                        gasMeter  gasPower
datetime                                  
2017-03-07 18:00:00  1599.792000  0.000539
2017-03-07 18:00:10  1599.792539  0.000539
2017-03-07 18:00:20  1599.793078  0.000539
2017-03-07 18:00:30  1599.793617  0.000539
2017-03-07 18:00:40  1599.794156  0.000539
2017-03-07 18:00:50  1599.794694  0.000539
2017-03-07 18:01:00  1599.795233  0.000539
2017-03-07 18:01:10  1599.795772  0.000539
2017-03-07 18:01:20  1599.796311  0.000539
2017-03-07 18:01:30  1599.796850  0.000539
2017-03-07 18:01:40  1599.797389  0.000539
2017-03-07 18:01:50  1599.797928  0.000539
2017-03-07 18:02:00  1599.798467  0.000539
2017-03-07 18:02:10  1599.799006  0.000539
2017-03-07 18:02:20  1599.799544  0.000539
2017-03-07 18:02:30  1599.800083  0.000539
2017-03-07 18:02:40  1599.800622  0.000539
2017-03-07 18:02:50  1599.801161  0.000539
2017-03-07 18:03:00  1599.801700  0.000539
2017-03-07 18:03:10  1599.802239  0.000539
2017-03-07 18:03:20  1599.802778  0.000539
2017-03-07 

In [13]:
print(smart.index[0])

2017-03-07 18:14:20


In [14]:
gas_selected=gas_final.ix[smart_final.index[0]:smart_final.index[-1]]

/opt/jupyterhub/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  """Entry point for launching an IPython kernel.


In [15]:
print(gas_selected)

                        gasMeter  gasPower
datetime                                  
2017-03-07 18:14:20  1599.838344  0.000539
2017-03-07 18:14:30  1599.838883  0.000539
2017-03-07 18:14:40  1599.839422  0.000539
2017-03-07 18:14:50  1599.839961  0.000539
2017-03-07 18:15:00  1599.840500  0.000539
2017-03-07 18:15:10  1599.841039  0.000539
2017-03-07 18:15:20  1599.841578  0.000539
2017-03-07 18:15:30  1599.842117  0.000539
2017-03-07 18:15:40  1599.842656  0.000539
2017-03-07 18:15:50  1599.843194  0.000539
2017-03-07 18:16:00  1599.843733  0.000539
2017-03-07 18:16:10  1599.844272  0.000539
2017-03-07 18:16:20  1599.844811  0.000539
2017-03-07 18:16:30  1599.845350  0.000539
2017-03-07 18:16:40  1599.845889  0.000539
2017-03-07 18:16:50  1599.846428  0.000539
2017-03-07 18:17:00  1599.846967  0.000539
2017-03-07 18:17:10  1599.847506  0.000539
2017-03-07 18:17:20  1599.848044  0.000539
2017-03-07 18:17:30  1599.848583  0.000539
2017-03-07 18:17:40  1599.849122  0.000539
2017-03-07 